# HIMARI Layer 2 - Transformer-A2C Training

**Run all cells in order to train the model with the fixed configuration.**

Data: Downloads from Google Drive automatically

## Step 1: Clone Repository & Install Dependencies

In [ ]:
# Clone the repository
!git clone https://github.com/nimallansa937/HIMARI-OPUS-TACTICAL-L2-V1.0.git
%cd HIMARI-OPUS-TACTICAL-L2-V1.0

In [ ]:
# Install dependencies
!pip install --upgrade pip -q
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q
!pip install numpy pandas scikit-learn scipy gdown tqdm -q
print("✅ Dependencies installed!")

## Step 2: Download BTC Data from Google Drive

In [ ]:
import gdown
import os

# Create data directory
os.makedirs("LAYER 2 TACTICAL HIMARI OPUS/data", exist_ok=True)

# Google Drive file ID from your share link
FILE_ID = "1_YMRsTCHjfsrqf63RI3xQ4jpehIsEaNW"
OUTPUT_PATH = "LAYER 2 TACTICAL HIMARI OPUS/data/btc_5min_2020_2024.pkl"

if not os.path.exists(OUTPUT_PATH):
    print("Downloading BTC data from Google Drive...")
    gdown.download(id=FILE_ID, output=OUTPUT_PATH, quiet=False)
    print(f"✅ Downloaded! File size: {os.path.getsize(OUTPUT_PATH) / 1e6:.1f} MB")
else:
    print(f"✅ Data file already exists: {OUTPUT_PATH}")

## Step 3: Verify Data

In [ ]:
import pickle
import numpy as np

data_path = "LAYER 2 TACTICAL HIMARI OPUS/data/btc_5min_2020_2024.pkl"
with open(data_path, 'rb') as f:
    raw_data = pickle.load(f)

print(f"✅ Data loaded successfully!")
print(f"Type: {type(raw_data)}")

if isinstance(raw_data, dict):
    for k, v in raw_data.items():
        if hasattr(v, 'shape'):
            print(f"  {k}: shape={v.shape}, dtype={v.dtype}")
        elif hasattr(v, '__len__'):
            print(f"  {k}: len={len(v)}")
elif hasattr(raw_data, 'shape'):
    print(f"Shape: {raw_data.shape}")

## Step 4: Setup Training Environment

In [ ]:
import sys
import logging

# Add source to path
sys.path.insert(0, 'LAYER 2 TACTICAL HIMARI OPUS')

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# Check GPU
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Step 5: Prepare Data & Create Environments

In [ ]:
from src.models.transformer_a2c import TransformerA2CConfig
from src.environment.transformer_a2c_env import TransformerA2CEnv, WalkForwardSplitter, TransformerEnvConfig

# Extract features and prices
if isinstance(raw_data, dict):
    if 'features' in raw_data and 'prices' in raw_data:
        features = raw_data['features']
        prices = raw_data['prices']
    elif 'data' in raw_data:
        features = raw_data['data']
        prices = raw_data.get('prices', raw_data.get('close', None))
    else:
        keys = list(raw_data.keys())
        features = raw_data[keys[0]]
        prices = raw_data[keys[1]] if len(keys) > 1 else None
else:
    features = raw_data
    prices = None

# Convert to numpy
features = np.array(features, dtype=np.float32)
if prices is not None:
    prices = np.array(prices, dtype=np.float32)
else:
    print("⚠️ No prices found, generating from first feature column")
    prices = features[:, 0] if features.ndim > 1 else features

print(f"Features shape: {features.shape}")
print(f"Prices shape: {prices.shape}")

# Create train/val/test splits
splitter = WalkForwardSplitter(
    features, 
    prices,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15
)

# Feature dimension
feature_dim = features.shape[1] if features.ndim > 1 else 1

# Create environments
env_config = TransformerEnvConfig(
    context_length=100,
    feature_dim=feature_dim,
)

train_env, val_env, test_env = splitter.create_envs(config=env_config)
print(f"✅ Environments created!")

## Step 6: Configure & Start Training

In [ ]:
from src.training.transformer_a2c_trainer import train_transformer_a2c

# Training config (WITH ALL FIXES APPLIED)
config = TransformerA2CConfig(
    input_dim=feature_dim,
    hidden_dim=256,
    num_heads=8,
    num_layers=4,
    context_length=100,
    entropy_coef=0.07,      # FIXED: increased from 0.01
    max_steps=100_000,
    val_frequency=25_000,
    patience=3,
)

print("=" * 70)
print("Training with FIXED configuration:")
print(f"  ✅ entropy_coef: {config.entropy_coef} (was 0.01)")
print(f"  ✅ Look-ahead bias: FIXED")
print(f"  ✅ Entropy decay: ENABLED (0.07 → 0.021)")
print(f"  ✅ Action distribution logging: ENABLED")
print("=" * 70)

In [ ]:
# START TRAINING
result = train_transformer_a2c(
    train_env=train_env,
    val_env=val_env,
    config=config,
    device="cuda",
    output_dir="./output/transformer_a2c",
    use_wandb=False,
)

if result:
    print("\n" + "=" * 70)
    print("✅ TRAINING COMPLETE!")
    print(f"Best checkpoint: {result['path']}")
    print(f"Best validation Sharpe: {result['val_sharpe']:.4f}")
    print("=" * 70)
else:
    print("Training completed but no best checkpoint found")

## Step 7: Download Checkpoints (Optional)

In [ ]:
# List checkpoints
!ls -la ./output/transformer_a2c/*.pt 2>/dev/null || echo "No checkpoints yet"

In [ ]:
# Compress checkpoints for download
!tar -czvf transformer_a2c_checkpoints.tar.gz ./output/transformer_a2c/
print("\n✅ Checkpoints compressed to: transformer_a2c_checkpoints.tar.gz")
print("Download this file from the Jupyter file browser.")